### Functions for automating model selection through cross-validation.

In [231]:
# Copyright (c) 2017 The Verde Developers.
# Distributed under the terms of the BSD 3-Clause License.
# SPDX-License-Identifier: BSD-3-Clause
#
# This code is part of the Fatiando a Terra project (https://www.fatiando.org)
#
"""
Functions for automating model selection through cross-validation.
"""
import warnings

import numpy as np
from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.base import clone
from sklearn.utils import check_random_state

from .base import check_fit_input, n_1d_arrays, BaseBlockCrossValidator
from .base.utils import score_estimator
from .coordinates import block_split
from .utils import dispatch, partition_by_sum


# Pylint doesn't like X, y scikit-learn argument names.
# pylint: disable=invalid-name,unused-argument


[docs]
class BlockShuffleSplit(BaseBlockCrossValidator):
    """
    Random permutation of spatial blocks cross-validator.

    Yields indices to split data into training and test sets. Data are first
    grouped into rectangular blocks of size given by the *spacing* argument.
    Alternatively, blocks can be defined by the number of blocks in each
    dimension using the *shape* argument instead of *spacing*. The blocks are
    then split into testing and training sets randomly.

    The proportion of blocks assigned to each set is controlled by *test_size*
    and/or *train_size*. However, the total amount of actual data points in
    each set could be different from these values since blocks can have
    a different number of data points inside them. To guarantee that the
    proportion of actual data is as close as possible to the proportion of
    blocks, this cross-validator generates an extra number of splits and
    selects the one with proportion of data points in each set closer to the
    desired amount [Valavi_etal2019]_. The number of balancing splits per
    iteration is controlled by the *balancing* argument.

    This cross-validator is preferred over
    :class:`sklearn.model_selection.ShuffleSplit` for spatial data to avoid
    overestimating cross-validation scores. This can happen because of the
    inherent autocorrelation that is usually associated with this type of data
    (points that are close together are more likely to have similar values).
    See [Roberts_etal2017]_ for an overview of this topic.

    .. note::

        Like :class:`sklearn.model_selection.ShuffleSplit`, this
        cross-validator cannot guarantee that all folds will be different,
        although this is still very likely for sizeable datasets.

    Parameters
    ----------
    spacing : float, tuple = (s_north, s_east), or None
        The block size in the South-North and West-East directions,
        respectively. A single value means that the spacing is equal in both
        directions. If None, then *shape* **must be provided**.
    shape : tuple = (n_north, n_east) or None
        The number of blocks in the South-North and West-East directions,
        respectively. If None, then *spacing* **must be provided**.
    n_splits : int, default 10
        Number of re-shuffling & splitting iterations.
    test_size : float, int, None, default=None
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. If ``train_size`` is also None, it will
        be set to 0.1.
    train_size : float, int, or None, default=None
        If float, should be between 0.0 and 1.0 and represent the
        proportion of the dataset to include in the train split. If
        int, represents the absolute number of train samples. If None,
        the value is automatically set to the complement of the test size.
    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.
    balancing : int
        The number of splits generated per iteration to try to balance the
        amount of data in each set so that *test_size* and *train_size* are
        respected. If 1, then no extra splits are generated (essentially
        disabling the balancing). Must be >= 1.

    See also
    --------
    train_test_split : Split a dataset into a training and a testing set.
    cross_val_score : Score an estimator/gridder using cross-validation.

    Examples
    --------

    >>> from verde import grid_coordinates
    >>> import numpy as np
    >>> # Make a regular grid of data points
    >>> coords = grid_coordinates(region=(0, 3, -10, -7), spacing=1)
    >>> # Need to convert the coordinates into a feature matrix
    >>> X = np.transpose([i.ravel() for i in coords])
    >>> shuffle = BlockShuffleSplit(spacing=1.5, n_splits=3, random_state=0)
    >>> # These are the 1D indices of the points belonging to each set
    >>> for train, test in shuffle.split(X):
    ...     print("Train: {} Test: {}".format(train, test))
    Train: [ 0  1  2  3  4  5  6  7 10 11 14 15] Test: [ 8  9 12 13]
    Train: [ 2  3  6  7  8  9 10 11 12 13 14 15] Test: [0 1 4 5]
    Train: [ 0  1  4  5  8  9 10 11 12 13 14 15] Test: [2 3 6 7]
    >>> # A better way to visualize this is to create a 2D array and put
    >>> # "train" or "test" in the corresponding locations.
    >>> shape = coords[0].shape
    >>> mask = np.full(shape=shape, fill_value="     ")
    >>> for iteration, (train, test) in enumerate(shuffle.split(X)):
    ...     # The index needs to be converted to 2D so we can index our matrix.
    ...     mask[np.unravel_index(train, shape)] = "train"
    ...     mask[np.unravel_index(test, shape)] = " test"
    ...     print("Iteration {}:".format(iteration))
    ...     print(mask)
    Iteration 0:
    [['train' 'train' 'train' 'train']
     ['train' 'train' 'train' 'train']
     [' test' ' test' 'train' 'train']
     [' test' ' test' 'train' 'train']]
    Iteration 1:
    [[' test' ' test' 'train' 'train']
     [' test' ' test' 'train' 'train']
     ['train' 'train' 'train' 'train']
     ['train' 'train' 'train' 'train']]
    Iteration 2:
    [['train' 'train' ' test' ' test']
     ['train' 'train' ' test' ' test']
     ['train' 'train' 'train' 'train']
     ['train' 'train' 'train' 'train']]

    """

    def __init__(
        self,
        spacing=None,
        shape=None,
        n_splits=10,
        test_size=0.1,
        train_size=None,
        random_state=None,
        balancing=10,
    ):
        super().__init__(spacing=spacing, shape=shape, n_splits=n_splits)
        if balancing < 1:
            raise ValueError(
                "The *balancing* argument must be >= 1. To disable balancing, use 1."
            )
        self.test_size = test_size
        self.train_size = train_size
        self.random_state = random_state
        self.balancing = balancing

    def _iter_test_indices(self, X=None, y=None, groups=None):
        """
        Generates integer indices corresponding to test sets.

        Runs several iterations until a split is found that yields blocks with
        the right amount of data points in it.

        Parameters
        ----------
        X : array-like, shape (n_samples, 2)
            Columns should be the easting and northing coordinates of data
            points, respectively.
        y : array-like, shape (n_samples,)
            The target variable for supervised learning problems. Always
            ignored.
        groups : array-like, with shape (n_samples,), optional
            Group labels for the samples used while splitting the dataset into
            train/test set. Always ignored.

        Yields
        ------
        test : ndarray
            The testing set indices for that split.

        """
        labels = block_split(
            coordinates=(X[:, 0], X[:, 1]),
            spacing=self.spacing,
            shape=self.shape,
            region=None,
            adjust="spacing",
        )[1]
        block_ids = np.unique(labels)
        # Generate many more splits so that we can pick and choose the ones
        # that have the right balance of training and testing data.
        shuffle = ShuffleSplit(
            n_splits=self.n_splits * self.balancing,
            test_size=self.test_size,
            train_size=self.train_size,
            random_state=self.random_state,
        ).split(block_ids)
        for _ in range(self.n_splits):
            test_sets, balance = [], []
            for _ in range(self.balancing):
                # This is a false positive in pylint which is why the warning
                # is disabled at the top of this file:
                # https://github.com/PyCQA/pylint/issues/1830
                # pylint: disable=stop-iteration-return
                train_blocks, test_blocks = next(shuffle)
                # pylint: enable=stop-iteration-return
                train_points = np.where(np.isin(labels, block_ids[train_blocks]))[0]
                test_points = np.where(np.isin(labels, block_ids[test_blocks]))[0]
                # The proportion of data points assigned to each group should
                # be close the proportion of blocks assigned to each group.
                balance.append(
                    abs(
                        train_points.size / test_points.size
                        - train_blocks.size / test_blocks.size
                    )
                )
                test_sets.append(test_points)
            best = np.argmin(balance)
            yield test_sets[best]



[docs]
class BlockKFold(BaseBlockCrossValidator):
    """
    K-Folds over spatial blocks cross-validator.

    Yields indices to split data into training and test sets. Data are first
    grouped into rectangular blocks of size given by the *spacing* argument.
    Alternatively, blocks can be defined by the number of blocks in each
    dimension using the *shape* argument instead of *spacing*. The blocks are
    then split into testing and training sets iteratively along k folds of the
    data (k is given by *n_splits*).

    By default, the blocks are split into folds in a way that makes each fold
    have approximately the same number of data points. Sometimes this might not
    be possible, which can happen if the number of splits is close to the
    number of blocks. In these cases, each fold will have the same number of
    blocks, regardless of how many data points are in each block. This
    behaviour can also be disabled by setting ``balance=False``.

    Shuffling the blocks prior to splitting is strongly encouraged. Not
    shuffling will essentially lead to the creation of *n_splits* large blocks
    since blocks are spatially adjacent when not shuffled. The default
    behaviour is not to shuffle for compatibility with similar cross-validators
    in scikit-learn.

    This cross-validator is preferred over
    :class:`sklearn.model_selection.KFold` for spatial data to avoid
    overestimating cross-validation scores. This can happen because of the
    inherent autocorrelation that is usually associated with this type of data
    (points that are close together are more likely to have similar values).
    See [Roberts_etal2017]_ for an overview of this topic.

    Parameters
    ----------
    spacing : float, tuple = (s_north, s_east), or None
        The block size in the South-North and West-East directions,
        respectively. A single value means that the spacing is equal in both
        directions. If None, then *shape* **must be provided**.
    shape : tuple = (n_north, n_east) or None
        The number of blocks in the South-North and West-East directions,
        respectively. If None, then *spacing* **must be provided**.
    n_splits : int
        Number of folds. Must be at least 2.
    shuffle : bool
        Whether to shuffle the data before splitting into batches.
    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.
    balance : bool
        Whether or not to split blocks into fold with approximately equal
        number of data points. If False, each fold will have the same number of
        blocks (which can have different number of data points in them).

    See also
    --------
    train_test_split : Split a dataset into a training and a testing set.
    cross_val_score : Score an estimator/gridder using cross-validation.

    Examples
    --------

    >>> from verde import grid_coordinates
    >>> import numpy as np
    >>> # Make a regular grid of data points
    >>> coords = grid_coordinates(region=(0, 3, -10, -7), spacing=1)
    >>> # Need to convert the coordinates into a feature matrix
    >>> X = np.transpose([i.ravel() for i in coords])
    >>> kfold = BlockKFold(spacing=1.5, n_splits=4)
    >>> # These are the 1D indices of the points belonging to each set
    >>> for train, test in kfold.split(X):
    ...     print("Train: {} Test: {}".format(train, test))
    Train: [ 2  3  6  7  8  9 10 11 12 13 14 15] Test: [0 1 4 5]
    Train: [ 0  1  4  5  8  9 10 11 12 13 14 15] Test: [2 3 6 7]
    Train: [ 0  1  2  3  4  5  6  7 10 11 14 15] Test: [ 8  9 12 13]
    Train: [ 0  1  2  3  4  5  6  7  8  9 12 13] Test: [10 11 14 15]
    >>> # A better way to visualize this is to create a 2D array and put
    >>> # "train" or "test" in the corresponding locations.
    >>> shape = coords[0].shape
    >>> mask = np.full(shape=shape, fill_value="     ")
    >>> for iteration, (train, test) in enumerate(kfold.split(X)):
    ...     # The index needs to be converted to 2D so we can index our matrix.
    ...     mask[np.unravel_index(train, shape)] = "train"
    ...     mask[np.unravel_index(test, shape)] = " test"
    ...     print("Iteration {}:".format(iteration))
    ...     print(mask)
    Iteration 0:
    [[' test' ' test' 'train' 'train']
     [' test' ' test' 'train' 'train']
     ['train' 'train' 'train' 'train']
     ['train' 'train' 'train' 'train']]
    Iteration 1:
    [['train' 'train' ' test' ' test']
     ['train' 'train' ' test' ' test']
     ['train' 'train' 'train' 'train']
     ['train' 'train' 'train' 'train']]
    Iteration 2:
    [['train' 'train' 'train' 'train']
     ['train' 'train' 'train' 'train']
     [' test' ' test' 'train' 'train']
     [' test' ' test' 'train' 'train']]
    Iteration 3:
    [['train' 'train' 'train' 'train']
     ['train' 'train' 'train' 'train']
     ['train' 'train' ' test' ' test']
     ['train' 'train' ' test' ' test']]

    For spatial data, it's often good to shuffle the blocks before assigning
    them to folds:

    >>> # Set the random_state to make sure we always get the same result.
    >>> kfold = BlockKFold(
    ...     spacing=1.5, n_splits=4, shuffle=True, random_state=123,
    ... )
    >>> for train, test in kfold.split(X):
    ...     print("Train: {} Test: {}".format(train, test))
    Train: [ 0  1  2  3  4  5  6  7  8  9 12 13] Test: [10 11 14 15]
    Train: [ 2  3  6  7  8  9 10 11 12 13 14 15] Test: [0 1 4 5]
    Train: [ 0  1  4  5  8  9 10 11 12 13 14 15] Test: [2 3 6 7]
    Train: [ 0  1  2  3  4  5  6  7 10 11 14 15] Test: [ 8  9 12 13]

    These should be the same splits as we got before but in a different order.
    This only happens because in this example we have the number of splits
    equal to the number of blocks (4). With real data the effects would be more
    dramatic.

    """

    def __init__(
        self,
        spacing=None,
        shape=None,
        n_splits=5,
        shuffle=False,
        random_state=None,
        balance=True,
    ):
        super().__init__(spacing=spacing, shape=shape, n_splits=n_splits)
        if n_splits < 2:
            raise ValueError(
                "Number of splits must be >=2 for BlockKFold. Given {}.".format(
                    n_splits
                )
            )
        self.shuffle = shuffle
        self.random_state = random_state
        self.balance = balance

    def _iter_test_indices(self, X=None, y=None, groups=None):
        """
        Generates integer indices corresponding to test sets.

        Parameters
        ----------
        X : array-like, shape (n_samples, 2)
            Columns should be the easting and northing coordinates of data
            points, respectively.
        y : array-like, shape (n_samples,)
            The target variable for supervised learning problems. Always
            ignored.
        groups : array-like, with shape (n_samples,), optional
            Group labels for the samples used while splitting the dataset into
            train/test set. Always ignored.

        Yields
        ------
        test : ndarray
            The testing set indices for that split.

        """
        labels = block_split(
            coordinates=(X[:, 0], X[:, 1]),
            spacing=self.spacing,
            shape=self.shape,
            region=None,
            adjust="spacing",
        )[1]
        block_ids = np.unique(labels)
        if self.n_splits > block_ids.size:
            raise ValueError(
                "Number of k-fold splits ({}) cannot be greater than the number of "
                "blocks ({}). Either decrease n_splits or increase the number of "
                "blocks.".format(self.n_splits, block_ids.size)
            )
        if self.shuffle:
            check_random_state(self.random_state).shuffle(block_ids)
        if self.balance:
            block_sizes = [np.isin(labels, i).sum() for i in block_ids]
            try:
                split_points = partition_by_sum(block_sizes, parts=self.n_splits)
                folds = np.split(np.arange(block_ids.size), split_points)
            except ValueError:
                warnings.warn(
                    "Could not balance folds to have approximately the same "
                    "number of data points. Dividing into folds with equal "
                    "number of blocks instead. Decreasing n_splits or increasing "
                    "the number of blocks may help.",
                    UserWarning,
                )
                folds = [i for _, i in KFold(n_splits=self.n_splits).split(block_ids)]
        else:
            folds = [i for _, i in KFold(n_splits=self.n_splits).split(block_ids)]
        for test_blocks in folds:
            test_points = np.where(np.isin(labels, block_ids[test_blocks]))[0]
            yield test_points
    


# pylint: enable=invalid-name,unused-argument


[docs]
def train_test_split(
    coordinates, data, weights=None, spacing=None, shape=None, **kwargs
):
    r"""
    Split a dataset into a training and a testing set for cross-validation.

    Similar to :func:`sklearn.model_selection.train_test_split` but is tuned to
    work on single- or multi-component spatial data with optional weights.

    If arguments *shape* or *spacing* are provided, will group the data by
    spatial blocks before random splitting (using
    :class:`verde.BlockShuffleSplit` instead of
    :class:`sklearn.model_selection.ShuffleSplit`). The argument *spacing*
    specifies the size of the spatial blocks. Alternatively, use *shape* to
    specify the number of blocks in each dimension.

    Extra keyword arguments will be passed to the cross-validation class. The
    exception is ``n_splits`` which is always 1.

    Grouping by spatial blocks is preferred over plain random splits for
    spatial data to avoid overestimating validation scores. This can happen
    because of the inherent autocorrelation that is usually associated with
    this type of data (points that are close together are more likely to have
    similar values). See [Roberts_etal2017]_ for an overview of this topic. To
    use spatial blocking, you **must provide** a *spacing* or *shape* argument
    (see below).

    Parameters
    ----------
    coordinates : tuple of arrays
        Arrays with the coordinates of each data point. Should be in the
        following order: (easting, northing, vertical, ...).
    data : array or tuple of arrays
        the data values of each data point. If the data has more than one
        component, *data* must be a tuple of arrays (one for each component).
    weights : none or array or tuple of arrays
        if not none, then the weights assigned to each data point. If more than
        one data component is provided, you must provide a weights array for
        each data component (if not none).
    spacing : float, tuple = (s_north, s_east), or None
        The spatial block size in the South-North and West-East directions,
        respectively. A single value means that the spacing is equal in both
        directions. If None, then *shape* must be provided in order to use
        spatial blocking.
    shape : tuple = (n_north, n_east) or None
        The number of blocks in the South-North and West-East directions,
        respectively. If None, then *spacing* must be provided in order to use
        spatial blocking.

    Returns
    -------
    train, test : tuples
        Each is a tuple = (coordinates, data, weights) generated by separating
        the input values randomly.

    See also
    --------
    cross_val_score : Score an estimator/gridder using cross-validation.
    BlockShuffleSplit : Random permutation of spatial blocks cross-validator.

    Examples
    --------

    To randomly split the data between training and testing sets:

    >>> import numpy as np
    >>> # Make some data
    >>> data = np.array([10, 30, 50, 70])
    >>> coordinates = (np.arange(4), np.arange(-4, 0))
    >>> train, test = train_test_split(coordinates, data, random_state=0)
    >>> # The training set:
    >>> print("coords:", train[0])
    coords: (array([3, 1, 0]), array([-1, -3, -4]))
    >>> print("data:", train[1])
    data: (array([70, 30, 10]),)
    >>> # The testing set:
    >>> print("coords:", test[0])
    coords: (array([2]), array([-2]))
    >>> print("data:", test[1])
    data: (array([50]),)

    If weights are given, they will also be split among the sets:

    >>> weights = np.array([4, 3, 2, 5])
    >>> train, test = train_test_split(
    ...     coordinates, data, weights, random_state=0,
    ... )
    >>> # The training set:
    >>> print("coords:", train[0])
    coords: (array([3, 1, 0]), array([-1, -3, -4]))
    >>> print("data:", train[1])
    data: (array([70, 30, 10]),)
    >>> print("weights:", train[2])
    weights: (array([5, 3, 4]),)
    >>> # The testing set:
    >>> print("coords:", test[0])
    coords: (array([2]), array([-2]))
    >>> print("data:", test[1])
    data: (array([50]),)
    >>> print("weights:", test[2])
    weights: (array([2]),)

    Data with multiple components can also be split:

    >>> data = (np.array([10, 30, 50, 70]), np.array([-70, -50, -30, -10]))
    >>> train, test = train_test_split(coordinates, data, random_state=0)
    >>> # The training set:
    >>> print("coords:", train[0])
    coords: (array([3, 1, 0]), array([-1, -3, -4]))
    >>> print("data:", train[1])
    data: (array([70, 30, 10]), array([-10, -50, -70]))
    >>> # The testing set:
    >>> print("coords:", test[0])
    coords: (array([2]), array([-2]))
    >>> print("data:", test[1])
    data: (array([50]), array([-30]))

    To split data grouped in spatial blocks:

    >>> from verde import grid_coordinates
    >>> # Make a regular grid of data points
    >>> coordinates = grid_coordinates(region=(0, 3, 4, 7), spacing=1)
    >>> data = np.arange(16).reshape((4, 4))
    >>> # We must specify the size of the blocks via the spacing argument.
    >>> # Blocks of 1.5 will split the domain into 4 blocks.
    >>> train, test = train_test_split(
    ...     coordinates, data, random_state=0, spacing=1.5,
    ... )
    >>> # The training set:
    >>> print("coords:", train[0][0], train[0][1], sep="\n")
    coords:
    [0. 1. 2. 3. 0. 1. 2. 3. 2. 3. 2. 3.]
    [4. 4. 4. 4. 5. 5. 5. 5. 6. 6. 7. 7.]
    >>> print("data:", train[1])
    data: (array([ 0,  1,  2,  3,  4,  5,  6,  7, 10, 11, 14, 15]),)
    >>> # The testing set:
    >>> print("coords:", test[0][0], test[0][1])
    coords: [0. 1. 0. 1.] [6. 6. 7. 7.]
    >>> print("data:", test[1])
    data: (array([ 8,  9, 12, 13]),)

    """
    args = check_fit_input(coordinates, data, weights, unpack=False)
    if spacing is None and shape is None:
        indices = np.arange(args[1][0].size)
        shuffle = ShuffleSplit(n_splits=1, **kwargs).split(indices)
    else:
        feature_matrix = np.transpose(n_1d_arrays(coordinates, 2))
        shuffle = BlockShuffleSplit(
            n_splits=1, spacing=spacing, shape=shape, **kwargs
        ).split(feature_matrix)
    split = next(shuffle)
    train, test = (tuple(select(i, index) for i in args) for index in split)
    return train, test



[docs]
def cross_val_score(
    estimator,
    coordinates,
    data,
    weights=None,
    cv=None,
    client=None,
    delayed=False,
    scoring=None,
):
    """
    Score an estimator/gridder using cross-validation.

    Similar to :func:`sklearn.model_selection.cross_val_score` but modified to
    accept spatial multi-component data with weights.

    By default, will use :class:`sklearn.model_selection.KFold` with
    ``n_splits=5`` and ``random_state=0`` to split the dataset. Any other
    cross-validation class from scikit-learn or Verde can be passed in through
    the *cv* argument.

    The score is calculated using the estimator/gridder's ``.score`` method by
    default. Alternatively, use the *scoring* parameter to specify a different
    scoring function (e.g., mean square error, mean absolute error, etc).

    Can optionally run in parallel using :mod:`dask`. To do this, use
    ``delayed=True`` to dispatch computations with :func:`dask.delayed` instead
    of running them. The returned scores will be "lazy" objects instead of the
    actual scores. To trigger the computation (which Dask will run in parallel)
    call the `.compute()` method of each score or :func:`dask.compute` with the
    entire list of scores.

    .. warning::

        The ``client`` parameter is deprecated and will be removed in Verde
        v2.0.0. Use ``delayed`` instead.

    Parameters
    ----------
    estimator : verde gridder
        Any verde gridder class that has the ``fit`` and ``score`` methods.
    coordinates : tuple of arrays
        Arrays with the coordinates of each data point. Should be in the
        following order: (easting, northing, vertical, ...).
    data : array or tuple of arrays
        the data values of each data point. If the data has more than one
        component, *data* must be a tuple of arrays (one for each component).
    weights : none or array or tuple of arrays
        if not none, then the weights assigned to each data point. If more than
        one data component is provided, you must provide a weights array for
        each data component (if not none).
    cv : None or cross-validation generator
        Any scikit-learn or Verde cross-validation generator. Defaults to
        :class:`sklearn.model_selection.KFold`.
    client : None or dask.distributed.Client
        **DEPRECATED:** This option is deprecated and will be removed in Verde
        v2.0.0. If None, then computations are run serially. Otherwise, should
        be a dask ``Client`` object. It will be used to dispatch computations
        to the dask cluster.
    delayed : bool
        If True, will use :func:`dask.delayed` to dispatch computations without
        actually executing them. The returned scores will be a list of delayed
        objects. Call `.compute()` on each score or :func:`dask.compute` on the
        entire list to trigger the actual computations.
    scoring : None, str, or callable
        A scoring function (or name of a function) known to scikit-learn. See
        the description of *scoring* in
        :func:`sklearn.model_selection.cross_val_score` for details. If None,
        will fall back to the estimator's ``.score`` method.

    Returns
    -------
    scores : array
        Array of scores for each split of the cross-validation generator. If
        *delayed*, will be a list of Dask delayed objects (see the *delayed*
        option). If *client* is not None, then the scores will be futures.

    See also
    --------
    train_test_split : Split a dataset into a training and a testing set.
    BlockShuffleSplit : Random permutation of spatial blocks cross-validator.

    Examples
    --------

    As an example, we can score :class:`verde.Trend` on data that actually
    follows a linear trend.

    >>> from verde import grid_coordinates, Trend
    >>> coords = grid_coordinates((0, 10, -10, -5), spacing=0.1)
    >>> data = 10 - coords[0] + 0.5*coords[1]
    >>> model = Trend(degree=1)

    In this case, the model should perfectly predict the data and R² scores
    should be equal to 1.

    >>> scores = cross_val_score(model, coords, data)
    >>> print(', '.join(['{:.2f}'.format(score) for score in scores]))
    1.00, 1.00, 1.00, 1.00, 1.00

    There are 5 scores because the default cross-validator is
    :class:`sklearn.model_selection.KFold` with ``n_splits=5``.

    To calculate the score with a different metric, use the *scoring* argument:

    >>> scores = cross_val_score(
    ...     model, coords, data, scoring="neg_mean_squared_error",
    ... )
    >>> print(', '.join(['{:.2f}'.format(-score) for score in scores]))
    0.00, 0.00, 0.00, 0.00, 0.00

    In this case, we calculated the (negative) mean squared error (MSE) which
    measures the distance between test data and predictions. This way, 0 is the
    best possible value meaning that the data and prediction are the same. The
    "neg" part indicates that this is the negative mean square error. This is
    required because scikit-learn assumes that higher scores are always treated
    as better (which is the opposite for MSE). For display, we take the
    negative of the score to get the actual MSE.

    We can use different cross-validators by assigning them to the ``cv``
    argument:

    >>> from sklearn.model_selection import ShuffleSplit
    >>> # Set the random state to get reproducible results
    >>> cross_validator = ShuffleSplit(n_splits=3, random_state=0)
    >>> scores = cross_val_score(model, coords, data, cv=cross_validator)
    >>> print(', '.join(['{:.2f}'.format(score) for score in scores]))
    1.00, 1.00, 1.00

    Often, spatial data are autocorrelated (points that are close together are
    more likely to have similar values), which can cause cross-validation with
    random splits to overestimate the prediction accuracy [Roberts_etal2017]_.
    To account for the autocorrelation, we can split the data in blocks rather
    than randomly with :class:`verde.BlockShuffleSplit`:

    >>> from verde import BlockShuffleSplit
    >>> # spacing controls the size of the spatial blocks
    >>> cross_validator = BlockShuffleSplit(
    ...     spacing=2, n_splits=3, random_state=0
    ... )
    >>> scores = cross_val_score(model, coords, data, cv=cross_validator)
    >>> print(', '.join(['{:.2f}'.format(score) for score in scores]))
    1.00, 1.00, 1.00

    We didn't see a difference here since our model and data are perfect. See
    :ref:`model_evaluation` for an example with real data.

    If using many splits, we can speed up computations by running them in
    parallel with Dask:

    >>> cross_validator = ShuffleSplit(n_splits=10, random_state=0)
    >>> scores_delayed = cross_val_score(
    ...     model, coords, data, cv=cross_validator, delayed=True
    ... )
    >>> # The scores are delayed objects.
    >>> # To actually run the computations, call dask.compute
    >>> import dask
    >>> scores = dask.compute(*scores_delayed)
    >>> print(', '.join(['{:.2f}'.format(score) for score in scores]))
    1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00

    Note that you must have enough RAM to fit multiple models simultaneously.
    So this is best used when fitting several smaller models.

    """
    if client is not None:
        warnings.warn(
            "The 'client' parameter of 'verde.cross_val_score' is deprecated "
            "and will be removed in Verde 2.0.0. "
            "Use the 'delayed' parameter instead.",
            FutureWarning,
        )
    coordinates, data, weights = check_fit_input(
        coordinates, data, weights, unpack=False
    )
    if cv is None:
        cv = KFold(shuffle=True, random_state=0, n_splits=5)
    feature_matrix = np.transpose(n_1d_arrays(coordinates, 2))
    fit_args = (coordinates, data, weights)
    scores = []
    for train_index, test_index in cv.split(feature_matrix):
        # Clone the estimator to avoid fitting the same object simultaneously
        # when delayed=True.
        score = dispatch(fit_score, client=client, delayed=delayed)(
            clone(estimator),
            tuple(select(i, train_index) for i in fit_args),
            tuple(select(i, test_index) for i in fit_args),
            scoring,
        )
        scores.append(score)
    if not delayed and client is None:
        scores = np.asarray(scores)
    return scores



def fit_score(estimator, train_data, test_data, scoring):
    """
    Fit an estimator on the training data and then score it on the testing data
    """
    estimator.fit(*train_data)
    if scoring is None:
        score = estimator.score(*test_data)
    else:
        score = score_estimator(scoring, estimator, *test_data)
    return score


def select(arrays, index):
    """
    Index each array in a tuple of arrays.

    If the arrays tuple contains a ``None``, the entire tuple will be returned
    as is.

    Parameters
    ----------
    arrays : tuple of arrays
    index : array
        An array of indices to select from arrays.

    Returns
    -------
    indexed_arrays : tuple of arrays

    Examples
    --------

    >>> import numpy as np
    >>> select((np.arange(5), np.arange(-3, 2, 1)), [1, 3])
    (array([1, 3]), array([-2,  0]))
    >>> select((None, None, None, None), [1, 2])
    (None, None, None, None)

    """
    if arrays is None or any(i is None for i in arrays):
        return arrays
    return tuple(i.ravel()[index] for i in arrays)

ImportError: attempted relative import with no known parent package

In [1]:
import verde as vd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import my_funcs as f

import pyproj
from shapely import geometry

In [2]:
%matplotlib widget

In [3]:
gama_cols = 'KB DATA BARO UB THB COSMICO CTB UUP ALTURA KPERC eU eTH CTEXP UTHRAZAO X Y UKRAZAO MDT THKRAZAO LIVE_TIME CTCOR KCOR THCOR UCOR HORA GPSALT LATITUDE FIDUCIAL TEMP LONGITUDE'.split(" ")

gLine_1105 = f.pd.read_csv(f.gdb+'xyz/1105_XYZ/1105_GamaLine.XYZ',
                         names=gama_cols,
                         delim_whitespace=True,
                         skiprows=11,
                         usecols=["X","Y","LATITUDE","LONGITUDE",
                                  "KPERC","eU","eTH","CTCOR","MDT"])

In [4]:
data_list = f.get_region('25k','SF23_Y_A_III_4_SW','gama_line_1105.csv')

# --- Iniciando seleção de área de estudo


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


1 folha cartográfica selecionada

# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['KPERC', 'eU', 'eTH', 'UTHRAZAO', 'UKRAZAO', 'MDT', 'THKRAZAO', 'CTCOR']
lista de atributos geograficos = ['LATITUDE', 'LONGITUDE']
lista de atributos projetados = ['X', 'Y']

# --- Início da iteração entre as folhas cartográficas #
# Folha de código: SF23_Y_A_III_4_SW
 Atualizando dados brutos em dic_cartas['raw_data']
 com 4778 pontos


In [5]:
print(f'Chaves do dicionario principal: {list(data_list[0].keys())}')
dic_cartas = data_list[0]

Chaves do dicionario principal: ['region', 'region_proj', 'raw_data', 'interpolado', 'scores', 'lito_geof', 'mean_score']


In [6]:
raw_data = dic_cartas['raw_data']['SF23_Y_A_III_4_SW']

In [7]:
#
list(data_list[2].keys())

['geometry', 'region', 'region_proj']

In [8]:
# Lista de dados aerogeofisicos
list(data_list[3].keys())

['KPERC', 'eU', 'eTH', 'UTHRAZAO', 'UKRAZAO', 'MDT', 'THKRAZAO', 'CTCOR']

In [9]:
coordinates = (raw_data.X.values, raw_data.Y.values)
coordinates

(array([319889.75, 319887.33, 319885.85, ..., 332398.19, 332404.87,
        332409.81]),
 array([7524730.34, 7524644.83, 7524559.28, ..., 7524555.55, 7524630.4 ,
        7524705.04]))

In [10]:
list(raw_data.columns)

['KPERC',
 'eU',
 'eTH',
 'UTHRAZAO',
 'X',
 'Y',
 'UKRAZAO',
 'MDT',
 'THKRAZAO',
 'CTCOR',
 'LATITUDE',
 'LONGITUDE']

In [11]:
list_titles =['KPERC', 'eU', 'eTH', 'MDT', 'CTCOR']

In [12]:
df_describe = raw_data.describe(percentiles= [0.005,0.1,0.25,0.5,0.75,0.995])

In [13]:
df_describe.T

,count,mean,std,min,0.5%,10%,25%,50%,75%,99.5%,max
KPERC,4778.0,1.545858e+00,0.702459,-3.040000e-01,2.688850e-01,7.657000e-01,1.052000e+00,1.408000e+00,1.941000e+00,3.833610e+00,4.666000e+00
eU,4778.0,1.683811e+00,0.783064,-4.720000e-01,-7.372500e-02,7.180000e-01,1.154000e+00,1.637000e+00,2.168000e+00,3.949415e+00,5.705000e+00
eTH,4778.0,1.519194e+01,6.253666,1.918000e+00,4.159125e+00,8.072100e+00,1.052450e+01,1.431700e+01,1.877250e+01,3.817771e+01,4.684200e+01
UTHRAZAO,4778.0,1.172641e-01,0.035828,2.200000e-02,4.288500e-02,7.400000e-02,9.200000e-02,1.150000e-01,1.390000e-01,2.520000e-01,3.080000e-01
X,4778.0,3.261858e+05,3720.867396,3.198610e+05,3.198731e+05,3.208840e+05,3.228830e+05,3.263643e+05,3.293857e+05,3.323935e+05,3.324175e+05
Y,4778.0,7.517789e+06,4020.289579,7.510778e+06,7.510849e+06,7.512177e+06,7.514327e+06,7.517831e+06,7.521251e+06,7.524688e+06,7.524761e+06
UKRAZAO,4778.0,1.234403e+00,0.606975,1.810000e-01,3.215400e-01,6.287000e-01,8.182500e-01,1.094000e+00,1.487750e+00,3.735840e+00,4.593000e+00
MDT,4778.0,7.995922e+02,123.077207,6.172800e+02,6.203197e+02,6.565370e+02,6.963550e+02,7.790750e+02,8.833875e+02,1.148144e+03,1.178110e+03
THKRAZAO,4778.0,1.104368e+01,5.557685,3.138000e+00,3.669965e+00,5.600400e+00,7.222000e+00,9.510000e+00,1.364200e+01,3.465758e+01,4.754500e+01
CTCOR,4778.0,7.554712e+00,3.108047,9.160000e-01,2.196195e+00,3.830100e+00,5.182000e+00,7.146000e+00,9.521000e+00,1.751467e+01,2.060900e+01


In [14]:
data = raw_data

In [15]:
print(f"Valores negativos no canal do K: {data.query('KPERC < 0')['KPERC'].count()}")
print(f"Valores negativos no canal do U: {data.query('eU < 0')['eU'].count()}")

Valores negativos no canal do K: 2
Valores negativos no canal do U: 43


In [16]:
print(f"{(data.query('eU < 0')['eU'].count()/df_describe['eU']['count'])*100} % do eU apresentavam valores negativos")

print(f"{(data.query('KPERC < 0')['KPERC'].count()/df_describe['KPERC']['count'])*100} % do KPERC apresentavam valores negativos")

0.8999581414817915 % do eU apresentavam valores negativos
0.041858518208455424 % do KPERC apresentavam valores negativos


In [17]:
data.loc[data.KPERC < 0, 'KPERC'] = 0
data.loc[data.eU < 0, 'eU'] = 0

In [18]:
print(data.query('KPERC < 0')['KPERC'].count())
print(data.query('eU < 0')['eU'].count())

0
0


In [19]:
data_descrito = data.describe(percentiles= [0.01,0.1,0.25,0.5,0.75,0.995])

In [20]:
print(f"Lista de atributos: {list_titles}")

# LISTA DE TITULOS 
titles =   ['K (%)',
            'U (ppm)',
            'Th (ppm)',
            'MDT (m)',
            'Contagem Total (μR/h)'
            ]


# DICIONARIO DE TITULOS
dic_titles = {}
for f, t in zip(list_titles, titles):
    dic_titles[f] = t

dic_titles

Lista de atributos: ['KPERC', 'eU', 'eTH', 'MDT', 'CTCOR']


{'KPERC': 'K (%)',
 'eU': 'U (ppm)',
 'eTH': 'Th (ppm)',
 'MDT': 'MDT (m)',
 'CTCOR': 'Contagem Total (μR/h)'}

In [21]:
def raw_plot(data):
    fig, axs = plt.subplots(nrows = 3, ncols = 2, figsize = (9, 9))

    X,Y = data.X,data.Y

    for ax, f in zip(axs.flat, dic_titles):
        vmin=data_descrito[f]['1%']
        vmax=data_descrito[f]['99.5%']

        g = ax.scatter(c=data[f], x = X,
                                y = Y,
                                cmap = 'rainbow',
                                s    = 0.5,
                                vmax =vmax, vmin=vmin)

        fig.colorbar(g, ax = ax)
        ax.set_title(str(dic_titles[f]), size =9)
        ax.axis('scaled')

    fig.delaxes(axs[2][1])

    #plt.tight_layout()

In [22]:
raw_plot(data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
# Block Reduction
reducer_mean_490 = vd.BlockReduce(np.mean, spacing= 490)
reducer_mean_499 = vd.BlockReduce(np.mean, spacing= 499)
reducer_mean_500 = vd.BlockReduce(np.mean, spacing= 500)

In [24]:
# Block Reduction
reducer_median_490 = vd.BlockReduce(np.median, spacing= 490)
reducer_median_499 = vd.BlockReduce(np.median, spacing= 499)
reducer_median_500 = vd.BlockReduce(np.median, spacing= 500)

In [25]:
lista_blocks = [reducer_mean_490,reducer_mean_499,reducer_mean_500,
                reducer_median_490,reducer_median_499,reducer_median_500]

In [26]:
lista_blocks[0]

BlockReduce(reduction=<function mean at 0x7f36d80b13a0>, spacing=490)

In [27]:
# Ploting the Reduced data
fig, axs = plt.subplots(nrows = 2, ncols = 3, figsize = (16, 16),sharex='all',sharey='all')

for ax, f in zip(axs.flat, lista_blocks):
    b_coords, b_eU = f.filter(coordinates, data.eU)

    g = ax.scatter(c    = b_eU, x = b_coords[0],
                                y = b_coords[1],
                             cmap = 'rainbow',
                             s    = 15,
                             marker='o')
    fig.colorbar(g,ax=ax)
    ax.set_title(f,size=4)
    ax.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
reducer_mean_490 = vd.BlockReduce(np.mean, spacing= 490)
b_coords, b_eU = reducer_mean_490.filter(coordinates, data.eU)


In [29]:
spline = vd.Spline()

In [30]:
spline.fit(b_coords, b_eU)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Spline()

In [31]:
predicted = spline.predict(coordinates)

In [32]:
plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c=predicted,
            cmap='rainbow',
            s=1)
plt.axis('scaled')
plt.title('Valores preditos para os mesmos pontos amostrados')
plt.colorbar(label='eU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
residuals = data.eU - predicted

In [34]:
scale = vd.maxabs(residuals)

plt.figure(figsize=(7,7))

plt.scatter(coordinates[0],coordinates[1],
            c=residuals,
            cmap='RdBu_r',
            s=2,
            vmin=-scale,vmax=scale)
plt.axis('scaled')
plt.title('Residuals: Dado amostrado - Dado predito')
plt.colorbar(label='eU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
region = vd.get_region(coordinates)
grid_coords = vd.grid_coordinates(region, spacing = 100,pixel_register=True)

In [36]:
grid_eU = spline.predict(grid_coords)

In [37]:
plt.figure(figsize=(8,8))
plt.scatter(grid_coords[0], grid_coords[1],
            c=grid_eU,
            cmap='rainbow',
            s=2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
grid = spline.grid(spacing=100, data_names=['eU'])
grid

<xarray.Dataset>
Dimensions:   (northing: 137, easting: 126)
Coordinates:
  * easting   (easting) float64 3.199e+05 3.2e+05 ... 3.323e+05 3.324e+05
  * northing  (northing) float64 7.511e+06 7.511e+06 ... 7.524e+06 7.525e+06
Data variables:
    eU        (northing, easting) float64 1.56 1.562 1.552 ... 2.682 2.876 3.041
Attributes:
    metadata:  Generated by Spline()

In [39]:
plt.figure()
grid.eU.plot(cmap='rainbow')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(319812.0477933333, 332457.96387333336, 7510936.227647059, 7524603.99235294)

In [40]:
grid = vd.distance_mask(coordinates, maxdist=500, grid=grid)
grid

grid.eU.plot(figsize=(7,7),cmap='rainbow')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(319812.0477933333, 332457.96387333336, 7510936.227647059, 7524603.99235294)

In [41]:
chain = vd.Chain([
    ('trend',  vd.Trend(degree=1)),
    ('reduce', vd.BlockReduce(np.mean, spacing=490)),
    ('spline', vd.Spline())
])

In [42]:
data.head()

,KPERC,eU,eTH,UTHRAZAO,X,Y,UKRAZAO,MDT,THKRAZAO,CTCOR,LATITUDE,LONGITUDE
24796,0.609,0.012,2.221,0.131,319889.75,7524730.34,1.065,669.99,8.128,0.916,-22.373985,-46.749332
24797,0.736,0.327,4.787,0.129,319887.33,7524644.83,1.063,674.43,8.228,1.376,-22.374757,-46.749365
24798,0.822,0.334,6.456,0.129,319885.85,7524559.28,1.080,678.37,8.354,1.806,-22.375529,-46.749389
24799,0.792,0.594,5.054,0.131,319885.26,7524473.80,1.109,679.32,8.473,2.070,-22.376301,-46.749404
24800,0.559,1.134,4.266,0.133,319885.44,7524388.35,1.141,680.23,8.548,2.337,-22.377073,-46.749412


In [43]:
chain.fit(coordinates, data.eU)

grid = chain.grid(spacing=100, data_names=['eU'])
grid_mask = vd.distance_mask(coordinates, maxdist=500, grid=grid)

In [44]:
grid_mask

<xarray.Dataset>
Dimensions:   (northing: 141, easting: 127)
Coordinates:
  * easting   (easting) float64 3.199e+05 3.2e+05 ... 3.323e+05 3.324e+05
  * northing  (northing) float64 7.511e+06 7.511e+06 ... 7.525e+06 7.525e+06
Data variables:
    eU        (northing, easting) float64 1.468 1.439 1.396 ... 2.77 2.925 3.066
Attributes:
    metadata:  Generated by Chain(steps=[('trend', Trend(degree=1)),\n       ...

In [45]:
grid_mask.eU.plot(figsize=(8,8), cmap='rainbow')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(319811.2126984127, 332467.34730158735, 7510727.711107143, 7524810.478892857)

Model Validation

In [46]:
train, test = vd.train_test_split(coordinates, data.eU,
                                 test_size = 0.1)

plt.figure(figsize=(4,4))
plt.plot(train[0][0], train[0][1], '.b', markersize=4)
plt.plot(test[0][0], test[0][1], '.r', markersize=4)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(319233.21599999996, 333045.34400000004, 7510078.5055, 7525459.6845)

In [47]:
chain.fit(*train)
# *train   ---> chain.fit(train[0], train[1])

chain.score(*test)


0.3361951443029668

In [48]:
train, test = vd.train_test_split(coordinates, data.eU, test_size=0.2, spacing=100)

plt.figure()
plt.plot(train[0][0], train[0][1], '.b', markersize=3)
plt.plot(test[0][0], test[0][1], '.r', markersize=3)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(319233.21599999996, 333045.34400000004, 7510078.5055, 7525459.6845)

In [49]:
chain.fit(*train)

chain.score(*test)

0.2629089686435674

In [50]:
data = raw_data

In [52]:
chain = vd.Chain([
    ('trend',  vd.Trend(degree=1)),
    ('reduce', vd.BlockReduce(np.mean, spacing=490)),
    ('spline', vd.Spline())
])

#train, test = vd.train_test_split(coordinates, data.eU, test_size=0.2, spacing=100)


cv     = vd.BlockKFold(spacing=10,                       # tamanho dos blocos
                  n_splits=100,                       
                  shuffle=True)

scores = vd.cross_val_score(chain,
                            coordinates,
                            data.eU,
                            cv=cv)

plt.figure()
plt.hist(scores, bins ='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 2.,  6., 10., 15., 21., 19., 20.,  7.]),
 array([0.02542902, 0.09441212, 0.16339522, 0.23237832, 0.30136141,
        0.37034451, 0.43932761, 0.50831071, 0.5772938 ]),
 <BarContainer object of 8 artists>)

In [53]:
scores

array([0.42002922, 0.15248961, 0.02542902, 0.3157736 , 0.37985084,
       0.46659891, 0.19669016, 0.3485033 , 0.22803246, 0.52197869,
       0.46594864, 0.25998301, 0.46999373, 0.26471413, 0.41059854,
       0.24099869, 0.27281788, 0.10351116, 0.46535152, 0.31986357,
       0.44790197, 0.157718  , 0.20288876, 0.49170439, 0.53791865,
       0.32625065, 0.18065951, 0.23882942, 0.21062802, 0.28622893,
       0.33941583, 0.10683509, 0.30927516, 0.44993331, 0.46228452,
       0.41957028, 0.4187105 , 0.31792121, 0.53727821, 0.42260815,
       0.35716123, 0.50481709, 0.16554598, 0.39102208, 0.44715902,
       0.34151744, 0.34483042, 0.19098684, 0.5044101 , 0.47408967,
       0.38115123, 0.32461957, 0.31821751, 0.41569769, 0.40334401,
       0.43219477, 0.46933821, 0.5772938 , 0.44498773, 0.33465702,
       0.44908854, 0.42498983, 0.3340695 , 0.35639257, 0.34572359,
       0.32651672, 0.33294771, 0.18411798, 0.37701743, 0.19844597,
       0.54374749, 0.15424476, 0.40108165, 0.37655057, 0.26913

In [54]:
geof = grid_mask.to_dataframe()
geof.reset_index(inplace=True)

In [55]:
geof

,northing,easting,eU
0,7510777.65,319861.040000,1.468212
1,7510777.65,319960.694603,1.439001
2,7510777.65,320060.349206,1.395522
3,7510777.65,320160.003810,1.334454
4,7510777.65,320259.658413,1.253331
...,...,...,...
17902,7524760.54,332018.901587,2.505900
17903,7524760.54,332118.556190,2.622138
17904,7524760.54,332218.210794,2.770460
17905,7524760.54,332317.865397,2.924985
